In [ ]:
import os
import matplotlib.pyplot as plt
from epic_kitchens.gulp.visualisation import FlowVisualiser
from epic_kitchens.dataset.epic_dataset import EpicVideoDataset, EpicVideoFlowDataset, GulpVideoSegment
from epic_kitchens.gulp.adapter import EpicDatasetAdapter
from pathlib import Path
import pandas as pd 
import numpy as np

In [ ]:
fpha_action_file = '/4TB/aaron/First_Person_Action_Benchmark/action_object_info.txt'
fpha_data = pd.read_csv(fpha_action_file, delim_whitespace=True)
fpha_nouns = [x.replace('_', ' ') for x in fpha_data.object_name.unique()]

fpha_verbs = fpha_data.action_name
fpha_verbs = [x.split('_')[0] for x in fpha_verbs]
fpha_verbs = list(dict.fromkeys(fpha_verbs))

fpha_verb_noun = [x.split('_') for x in fpha_data.action_name]
fpha_verb_noun = [' '.join(x) for x in fpha_verb_noun]
print(fpha_verb_noun)

In [ ]:
epic_root = '/4TB/aaron/EPIC_KITCHENS_2018/'

epic_action_data =  os.path.join(epic_root, 'annotations', 'EPIC_train_action_labels.csv')
epic_action_data = pd.read_csv(epic_action_data)

epic_nouns = [x.replace(':', ' ') for x in epic_action_data.noun.unique()]

epic_verbs = epic_action_data.verb.unique()

epic_verb_noun = [x + ' ' + y.replace(':', ' ') for x,y in zip(epic_action_data.verb, epic_action_data.noun)]

In [ ]:
# print('FPHA NOUNS:', fpha_nouns, '\n')
# print('EPIC_NOUNS:', epic_nouns, '\n')
print('Noun Intersection:', np.intersect1d(fpha_nouns, epic_nouns), 
      '\'washing liquid (liquid soap)\'', 
      '\'can (soda can)\'') 

In [ ]:
# print('FPHA VERBS:', fpha_verbs, '\n')
# print('EPIC_VERBS:', epic_verbs, '\n')
print('Verb Intersection:', np.intersect1d(fpha_verbs, epic_verbs))
print('**Not the same kind of toast')

In [ ]:
# print('FPHA VERBS_NOUNS:', fpha_verb_noun, '\n')
# print('EPIC_VERBS_NOUNS:', epic_verb_noun, '\n')
fpha_and_epic_verb_nouns = list(np.intersect1d(fpha_verb_noun, epic_verb_noun))
fpha_and_epic_verb_nouns.append('put teabag')
fpha_and_epic_verb_nouns.append('close juice')
fpha_and_epic_verb_nouns.append('open juice')
fpha_and_epic_verb_nouns.append('pour juice')
fpha_and_epic_verb_nouns.append('pour liquid washing')
fpha_and_epic_verb_nouns.append('open can')
fpha_and_epic_verb_nouns.append('pour wine')

print('Verb + Noun Intersection:', fpha_and_epic_verb_nouns)


In [ ]:
unique, counts = np.unique(epic_verb_noun, return_counts=True)
epic_verb_noun_dict = dict(zip(unique, counts))
for vn in fpha_and_epic_verb_nouns:
    print(vn, epic_verb_noun_dict[vn])

In [ ]:
epic_object_data =  os.path.join(epic_root, 'annotations', 'EPIC_train_object_labels.csv')
epic_object_data = pd.read_csv(epic_object_data)
bb_list = epic_object_data.bounding_boxes.values.tolist()
sum_bb = 0
for i in bb_list:
    if i != '[]':
        sum_bb += 1
        
print(sum_bb)

In [ ]:
epic_object_nouns = [x.replace(':', ' ') for x in epic_object_data.noun.unique()]
epic_fpha_object_nouns = list(np.intersect1d(fpha_nouns, epic_object_nouns))
epic_fpha_object_nouns.append('washing liquid')
epic_fpha_object_nouns.append('cans')
print('Object Intersection:', epic_fpha_object_nouns)

In [ ]:
unique, counts = np.unique(epic_object_data.noun, return_counts=True)
epic_noun_dict = dict(zip(unique, counts))
for n in epic_fpha_object_nouns:
    print(n, epic_noun_dict[n])

In [ ]:
gulp_root = Path('/4TB/aaron/EPIC_KITCHENS_2018/epic_ar/data/processed/gulp')

class_type = 'verb+noun'
rgb_train = EpicVideoDataset(gulp_root / 'rgb_train', class_type)



In [ ]:
# for i, seg in enumerate(rgb_train.video_segments):
#     if seg['verb'] == 'squeeze' and seg['noun'] == 'sponge':
#         print(i)

In [ ]:
epic_seg = rgb_train.video_segments[749]
epic_frames = rgb_train.load_frames(put_spoon_seg)
epic_frames[30]

In [ ]:
from moviepy.editor import ImageSequenceClip
segment_clip = ImageSequenceClip([np.array(frame) for frame in epic_frames], fps=60)
segment_clip.ipython_display()
# segment_clip.write_videofile("squeeze_sponge_movie.mp4")

In [ ]:
from PIL import Image
def get_data_list(modality, dataset_dir):
    train_pairs = []
    test_pairs = []
    img_dir = os.path.join(dataset_dir, 'Video_files')
    skel_dir = os.path.join(dataset_dir, 'Hand_pose_annotation_v1')
    if modality == 'depth':
        img_type = 'png'
    else:
        img_type = 'jpeg'
    with open(os.path.join(dataset_dir, 'data_split_action_recognition.txt')) as f:
        cur_split = 'Training'
        lines = f.readlines()
        for l in lines:
            words = l.split()
            if(words[0] == 'Training' or words[0] == 'Test'):
                cur_split = words[0]
            else:
                path = l.split()[0]
                full_path = os.path.join(img_dir, path, modality)
                len_frame_idx = len([x for x in os.listdir(full_path)
                                    if os.path.join(full_path, x)])
                skeleton_path = os.path.join(skel_dir, path, 'skeleton.txt')
                skeleton_vals = np.loadtxt(skeleton_path)
                for i in range(len_frame_idx):
                    img_path = os.path.join(img_dir, path, modality, '%s_%04d.%s' %(modality, i, img_type))
                    skel_xyz = skeleton_vals[:, 1:].reshape(skeleton_vals.shape[0], -1)[i]
                    data_pair = (img_path, skel_xyz)
                    if cur_split == 'Training':
                        train_pairs.append(data_pair)
                    else:
                        test_pairs.append(data_pair)
    return train_pairs, test_pairs

root = '/4TB/aaron/First_Person_Action_Benchmark'
train_pairs, test_pairs = get_data_list('color', root)
file_name = [i for i,j in train_pairs]
train_file_name = [file.split('/') for file in file_name]
file_name = [i for i,j in test_pairs]
test_file_name = [file.split('/') for file in file_name]

fpha_frames = []
file_name = test_file_name
for idx in range(3705, 3778):

    subject = file_name[idx][5]
    action_name = file_name[idx][6]
    seq_idx = file_name[idx][7]
    frame_idx = int(file_name[idx][-1][-9:-5])

    sample = {
        'subject': subject,
        'action_name': action_name,
        'seq_idx': seq_idx,
        'frame_idx': frame_idx,
    }
    
    img_path = os.path.join(root, 'Video_files', sample['subject'],
                            sample['action_name'], sample['seq_idx'], 'color',
                            'color_{:04d}.jpeg'.format(sample['frame_idx']))
    fpha_frames.append(np.asarray(Image.open(img_path)))


In [ ]:
# from moviepy.editor import ImageSequenceClip
# segment_clip = ImageSequenceClip([np.array(frame) for frame in fpha_frames], fps=60)
# segment_clip.ipython_display()